In [70]:
import pandas as pd
from simplegmail import Gmail
import imaplib
import email
from email.message import EmailMessage
import smtplib


In [81]:
df = pd.read_pickle('type.pkl')


emails =  { "technology":"yashjoshi30052@gmail.com", 
                "News" : "yashjoshi30052@gmail.com", 
                "Job search":'gourav.dangi@blinkx.in', 
                "Editorial":"gourav.dangi@blinkx.in", 
                "Blog" : "yashjoshi30052@gmail.com",
                "Webinar" : "yashjoshi30052@gmail.com",
                "Bank Related": 'gourav.dangi@blinkx.in'}

df['email'] = df['type'].map(emails)

df.head()


,uid,to,from,subject,content,timestamp,type,confidence,type_confidence,urgency,urgency_confidence,sentiment,sentiment_confidence,email
0,16458,yashjoshi30052002@gmail.com,Google <no-reply@accounts.google.com>,security alert,image google fetchemail granted access g...,2025-04-10 10:21:16 UTC,technology,None,0.855329,major,0.516962,neutral,0.800549,yashjoshi30052@gmail.com
1,16459,yashjoshi30052002@gmail.com,redBus <greetings@travel.e-redbus.in>,yash plan festive journey :blossom:,redbus welcome plan bus journey new year...,2025-04-10 16:24:19 UTC+05:30,technology,None,0.340465,major,0.613233,neutral,0.634413,yashjoshi30052@gmail.com


In [83]:

IMAP_SERVER = 'imap.gmail.com'
SMTP_SERVER = 'smtp.gmail.com'
EMAIL_ACCOUNT = 'yashjoshi30052002@gmail.com'  
APP_PASSWORD = 'iuox okin ayza xtmb'  


def forward_email_by_uid(recipient_email, uid, urgency, sentiment):
    try:
        
        mail = imaplib.IMAP4_SSL(IMAP_SERVER)
        mail.login(EMAIL_ACCOUNT, APP_PASSWORD)
        mail.select('inbox')  

        result, data = mail.uid('fetch', uid, '(RFC822)')
        if result != 'OK':
            raise Exception(f"Failed to fetch email with UID {uid}")

        raw_email = data[0][1]
        original_msg = email.message_from_bytes(raw_email)

        mail.logout()

       
        subject = original_msg['Subject']
        sender = original_msg['From']

        body = ""
        if original_msg.is_multipart():
            for part in original_msg.walk():
                if part.get_content_type() == "text/plain" and not part.get('Content-Disposition'):
                    charset = part.get_content_charset()
                    body = part.get_payload(decode=True).decode(charset or 'utf-8')
                    break
        else:
            charset = original_msg.get_content_charset()
            body = original_msg.get_payload(decode=True).decode(charset or 'utf-8') if charset else original_msg.get_payload()

      
        forward_msg = EmailMessage()
        forward_msg['Subject'] = 'Fwd: ' + subject
        forward_msg['From'] = EMAIL_ACCOUNT
        forward_msg['To'] = recipient_email

        forward_body = f"""---------- Forwarded message ----------
From: {sender}
Subject: {subject}
Urgency : {urgency}
Mail Nature : Sentiment - {sentiment}

{body}
"""
        forward_msg.set_content(forward_body)

        with smtplib.SMTP(SMTP_SERVER, 587) as smtp:
            smtp.starttls()
            smtp.login(EMAIL_ACCOUNT, APP_PASSWORD)
            smtp.send_message(forward_msg)
            print(f"Email with UID {uid} forwarded successfully to {recipient_email}.")

    except Exception as e:
        print(f"An error occurred while processing UID {uid}: {e}")


for index, row in df.iterrows():
        recipient = row['email']
        email_uid = row['uid']
        urgency = row['urgency']
        sentiment= row['sentiment']
        forward_email_by_uid(recipient, email_uid, urgency, sentiment)

Email with UID 16458 forwarded successfully to yashjoshi30052@gmail.com.
Email with UID 16459 forwarded successfully to yashjoshi30052@gmail.com.
